In [1]:
import os
from pypdf import PdfReader
import pandas as pd
import re

In [ ]:
# Define the search term and the directory
search_terms = [
    ("Aktiva", r"((20\d{2}).*(20\d{2}))|((Gesch.{0,2}ftsjahr).*(Vorjahr))"), 
    ("Passiva", r"(20\d{2}).*(20\d{2})"), 
    ("Gewinn", "Verlust", r"Rechnung|GuV", r"(20\d{2}).*(20\d{2})")
]
base_dir = "../Geschaeftsberichte/"

ocr_required = [
    "Geschaeftsberichte/degewo AG/2017_degewo_degewo_Homepage.pdf",
    "Geschaeftsberichte/degewo AG/2018_degewo_Konzernlagebericht_und_Konzernabschluss_fuer_das_Geschaeftsjahr.pdf",
    "Geschaeftsberichte/degewo AG/2019_degewo_Konzernbericht_und_Gleicher_Lohn_fuer_gleiche_Arbeit.pdf",
    # "Geschaeftsberichte/degewo AG/degewo_Konzernlagebericht_Konzernabschluss_2022.pdf", # Tabellen von Interesse nur als Scan
    # "Geschaeftsberichte/degewo AG/201231_degewo_Konzernlagebericht_Konzernabschluss.pdf", # Tabellen von Interesse nur als Scan
]

# Initialize a list to store results
results = []

# Recursively search through the directory
for root, _, files in os.walk(base_dir):
    for file in files:
        if file.endswith(".pdf"):
            filepath = os.path.join(root, file)
            print(filepath)
            if filepath == "../Geschaeftsberichte/bb_2024_gesamt.pdf":
                continue
            if filepath[3:] in ocr_required:
                print(f"Skipping OCR required file: {filepath}")
                continue
            
            try:
                # Open the PDF file
                reader = PdfReader(filepath)
                for page_num, page in enumerate(reader.pages):
                    text = page.extract_text()
                    for terms in search_terms:
                        # Check if all terms in the tuple are found on the page
                        if all(
                            re.search(term, text, re.IGNORECASE) if isinstance(term, str) else False
                            for term in terms
                        ):
                            match = re.search(terms[0], text, re.IGNORECASE)
                            if match:
                                start_idx = match.start()
                                context = text[max(0, start_idx - 50):start_idx + 50]
                                results.append({
                                    "filepath": filepath,
                                    "page": page_num + 1,
                                    "type": terms[0],
                                    "context": context.strip()
                                })
            except Exception as e:
                print(f"Error reading {filepath}: {e}")

# Convert results to a DataFrame
df = pd.DataFrame(results, columns=["filepath", "page", "context"])

# Save the results to a CSV file
output_csv = "bilanz_search_results_stricter_regex2.csv"
df.to_csv(output_csv, index=False)

print(f"Search completed. Results saved to {output_csv}.")

../Geschaeftsberichte/bb_2024_gesamt.pdf
../Geschaeftsberichte/Berliner Bäder Betriebe/GB_BBB_Infra_2022_low.pdf
../Geschaeftsberichte/Berliner Bäder Betriebe/2023_Geschaeftsbericht_BBB.pdf
../Geschaeftsberichte/Berliner Bäder Betriebe/GB_BBB_Infra_2020.pdf
../Geschaeftsberichte/Berliner Bäder Betriebe/2020_Geschaeftsbericht_BBB.pdf
../Geschaeftsberichte/Berliner Bäder Betriebe/GB_BBB_Infra_2023.pdf
../Geschaeftsberichte/Berliner Bäder Betriebe/2021_Geschaeftsbericht_BBB.pdf
../Geschaeftsberichte/Berliner Bäder Betriebe/GB_BBB_Infra_2019.pdf
../Geschaeftsberichte/Berliner Bäder Betriebe/2022_Geschaeftsbericht_BBB.pdf
../Geschaeftsberichte/Berliner Bäder Betriebe/2019_Geschaeftsbericht_BBB.pdf
../Geschaeftsberichte/Berliner Bäder Betriebe/GB_BBB_Infra_2021.pdf
../Geschaeftsberichte/IBB/ibb_geschaeftsbericht_2017.pdf
../Geschaeftsberichte/IBB/ibb-jahresabschluss-und-lagebericht-2023.pdf
../Geschaeftsberichte/IBB/ibb_geschaeftsbericht_2020.pdf
../Geschaeftsberichte/IBB/ibb_geschaeftsberic

In [30]:
output_csv = "bilanz_search_results_stricter_regex2.csv"
df = pd.read_csv(output_csv)[['filepath', 'page']].drop_duplicates()

In [31]:
df.shape[0]

406

In [12]:
df.head()

,filepath,page
0,../Geschaeftsberichte/Berliner Bäder Betriebe/...,10
1,../Geschaeftsberichte/Berliner Bäder Betriebe/...,11
2,../Geschaeftsberichte/Berliner Bäder Betriebe/...,13
3,../Geschaeftsberichte/Berliner Bäder Betriebe/...,15
6,../Geschaeftsberichte/Berliner Bäder Betriebe/...,22


In [16]:
from pypdf import PdfReader, PdfWriter

# Initialize a PdfWriter object
output_pdf = "combined_pages.pdf"
pdf_writer = PdfWriter()

# Iterate through the rows of the dataframe
for _, row in df.iterrows():
    filepath = row['filepath']
    page_number = row['page'] - 1  # Convert to zero-based index
    try:
        # Read the PDF file
        reader = PdfReader(filepath)
        # Add the specified page to the writer
        pdf_writer.add_page(reader.pages[page_number])
    except Exception as e:
        print(f"Error processing {filepath} page {page_number + 1}: {e}")

# Write the combined PDF to a file
with open(output_pdf, "wb") as out_file:
    pdf_writer.write(out_file)

print(f"Combined PDF created: {output_pdf}")

Combined PDF created: combined_pages.pdf


In [13]:
df.to_csv("table_page_truth.csv", index=False)